# Noms du binome: Valiau, Descarpentries 

# Prénoms du binome: Virgile, Rémy

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe qui générera un mot de passe en prennant **8 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [ ]:
import numpy as np
from collections import Counter
from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

## Modèle monogramme (une lettre)
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [ ]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values

Q: Quelles sont les 5 lettres les plus représentées ?

In [ ]:
monogramme.sort_values(by=['frequency'], ascending=False).head(5)

Les 5 lettres les plus représentées sont E, S, A, N et T.

Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien "gérer le cas ou la probabilité est nulle).

In [ ]:
def entropie(freq):
    ent = 0
    for i in freq :
        if i != 0 :
            ent = ent + i*np.log2(i)
    return -ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [ ]:
print ('Entropie mot de passe de 8 lettres pour le modèle monogramme :', round(entropie(freq_mono)*8, 3)," bits")

Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [ ]:
t = time.time()

# TO DO
np.random.choice(letters_mono, size = (100000, 8), p = freq_mono)

t_100000 = time.time() - t
print('Temps nécessaire pour générer 100 000 MdP avec le modèle monogramme :', round(t_100000, 5), 's')


Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [ ]:
H1 = entropie(freq_mono)*8
minorant1 = round((2**H1)/4+1)
print('Pour le modèle monogramme : G >= ', "{:,.0f}".format(minorant1).replace(',', ' ').replace('.', ','))

Q: combien de temps cela prendra-t-il pour générer un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [ ]:
t = (t_100000/100000)*minorant1

# TO DO

print('Pour décoder un mot de passe avec le modèle monogramme, cela prendrait en moyenne', round(t)//60, ' min ', round(t)%60, ' sec')

## Modèle bigramme (deux lettres)

On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [ ]:
bigramme = read_csv('bigramme.csv',keep_default_na=False)
freq_bi = (bigramme['frequency']).values
letters_bi = (bigramme['letters']).values

Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [ ]:
bigramme.sort_values(by=['frequency'], ascending=False).head(5)

Les 5 couples de lettres les plus représentés sont ES, EN, DE, LE et NT.

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [ ]:
print ('Entropie mot de passe de 8 lettres pour le modèle bigramme :', round(entropie(freq_bi)*4, 3)," bits")

Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

Avec le modèle bigramme, on choisit 4 couples de lettres. Ainsi, la deuxième lettre du mot (notée Y) est dépendante de la première lettre (notée X). Or, l'entropie conjointe de X et de Y est inférieure à la somme des entropies de X et de Y : H(X,Y) < H(X) + H(Y). Dans le modèle des monogrammes, les 8 lettres du mot de passe sont choisies de manière indépendante. Ainsi X et Y sont indépendantes et l'inégalité précédente devient une égalité : H(X,Y) = H(X) + H(Y). Par conséquent l'entropie construite sur les monogrammes est plus grande que l'entropie construite sur les bigrammes.

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [ ]:
t = time.time()

# TO DO
np.random.choice(letters_bi, size = (100000, 4), p = freq_bi)

t_100000_1 = time.time() - t
print('Temps nécessaire pour générer 100 000 MdP avec le modèle bigramme :', round(t_100000_1, 5), 's')


Q: calculer le minorant de $G$ pour ce modèle

In [ ]:
# TO DO
H2 = entropie(freq_bi)*4
minorant2 = round((2**H2)/4+1)
print('Pour le modèle bigramme : G >= ', "{:,.0f}".format(minorant2).replace(',', ' ').replace('.', ','))

Q: combien de temps cela prendra-t-il si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [ ]:
t1 = (t_100000_1/100000)*minorant2

# TO DO

print('Pour décoder un mot de passe avec le modèle bigramme, cela prendrait en moyenne', round(t1)//60, ' min ', round(t1)%60, ' sec')

In [ ]:
print ('Rapport entre le nombre moyen d\'essais nécessaires pour chaque méthode :', round(minorant1/minorant2, 3))
print ('Différence d\'entropie entre les deux méthodes :', round(H1-H2, 3))

En utilisant un modèle plus évolué tel que le modèle bigramme, nous diminuons l'entropie du mot de passe d'environ 1,5 bits. Ainsi, le temps nécessaire pour décoder un mot de passe est quasiment divisé par 3 par rapport au modèle monogramme. En effet, nous avons en moyenne besoin de faire 3 fois plus d'essais avec le modèle monogramme.

## Tirage aléatoire

Q: si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [ ]:
# TO DO
print ('Entropie mot de passe de 8 lettres tirées aléatoirement :', round(entropie(np.ones(26)*(1/26))*8, 3)," bits")

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [ ]:
# TO DO
t = time.time()

# TO DO
np.random.choice(letters_mono, size = (100000, 8), p = None)

t_100000_3 = time.time() - t
print('Temps nécessaire pour générer 100 000 MdP avec un tirage aléatoire :', round(t_100000_3, 5), 's')


Q: calculer le minorant de $G$ pour ce modèle

In [ ]:
H3 = entropie(np.ones(26)*(1/26))*8
minorant3 = round((2**H3)/4+1)
print('Pour un tirage aléatoire : G >= ', "{:,.0f}".format(minorant3).replace(',', ' ').replace('.', ','))

Q: dans ce cas précis, quelle est la valeur exacte de $G$?

In [ ]:
print('Valeur exacte de G :', "{:,.0f}".format(26**8/2).replace(',', ' ').replace('.', ','))

Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [ ]:
t3 = (t_100000_3/100000)*minorant3

# TO DO

print('Pour décoder un mot de passe avec un tirage aléatoire, cela prendrait en moyenne', round(t3)//60, ' min ', round(t3)%60, ' sec')

In [ ]:
print ('Rapport entre le nombre moyen d\'essais nécessaires pour chaque méthode :', round(minorant3/minorant2, 3))
print ('Différence d\'entropie entre les deux méthodes :', round(H3-H2, 3))

En tirant aléatoirement les 8 lettres du mots de passe, on augmente l'entropie de 7,5 bits par rapport au modèle bigramme. Ansi, on a multiplié le nombre d'essais nécessaire par un facteur d'ordre de grandeur 100. Ainsi, le temps de décodage explose (plusieurs heures). Pourtant, générer 100 000 mots de passe avec un tirage aléatoire est environ 2 à 3 fois plus rapide que générer ce même nombre de MdP avec un modèle bigramme ou monogramme.

## Attaque pratique

Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur:** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [ ]:
text_hugo = open("texteFrancais.txt","r")
str_hugo = str(text_hugo.read())

# On remplace des lettres avec accent avec des lettres sans accent
str_hugo = str_hugo.replace("Â", "A")
#str_hugo = str_hugo.replace("Ã", "A")
str_hugo = str_hugo.replace("Ù", "U")
str_hugo = str_hugo.replace("Ô", "O")
str_hugo = str_hugo.replace("Ã‚", "A")
str_hugo = str_hugo.replace("Ã”", "O")
str_hugo = str_hugo.replace("Ã™", "U")
size_txt = len(str_hugo)

Counter(str_hugo)

### Construction du dictionnaire

In [ ]:
bi = bigramme.sort_values(by=['frequency'], ascending=False)['letters']
pb = bigramme.sort_values(by=['frequency'], ascending=False)['frequency']
mdp1 = [x+y for x in bi for y in bi]
prb = [x*y for x in pb for y in pb]
tab_passwd = pd.DataFrame(np.array([mdp1, prb]).T, columns = ['mdp', 'proba'])
tab_passwd[['proba']] = tab_passwd[['proba']].astype(float)
tab_passwd = tab_passwd.drop_duplicates()
tab_passwd = tab_passwd.sort_values(by=['proba'], ascending=False)
tab_passwd.head(10)

### Attaque sur 1 000 mots de passe

In [ ]:
def appartient (x, liste):
    for i in range (len(liste)):
        if x == liste[i]:
            return i+1
    return False

essais = []
for i in range (1000):
    idx_rand = np.random.randint(size_txt-4)
    psswd = str_hugo[idx_rand:idx_rand+4]    
    essais.append(appartient(psswd, (tab_passwd['mdp']).values))

print('Nombre moyens d\'appel au dictionnaire nécessaire :', round(np.mean(essais))) 

In [ ]:
# Comparaison avec la valeur de G
# To do
H = entropie(freq_bi)*2
minorant4 = round((2**H)/4+1)
print('Pour le modèle bigramme avec un mot de passe de 4 lettres : G >= ', minorant4)

Le nombre d'essais moyen minimal théorique est environ 1,5 fois plus faible que le nombre d'essais moyen obtenu en pratique.

In [ ]:
H = entropie(np.ones(26)*(1/26))*4
minorant5 = round((2**H)/4+1)
print('En tirant 4 lettres de manière aléatoire : G >= ', "{:,.0f}".format(minorant5).replace(',', ' ').replace('.', ','))

En tirant 4 lettres de manière aléatoire, on a besoin d'environ 13 fois plus d'essais qu'avec le modèle bigramme. Le modèle bigramme est assez efficace pour décoder un mot de passe de 4 lettres puisque en théorie, le nombre minimal d'essais en moyenne est de 8 607.

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
- Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

Pour le défenseur : 

- Ne pas utiliser un mot unique du dictionnaire
- Utiliser des combinaisons de lettres peu communes (exemple : 'YW', 'HX' plutôt que 'ES', 'TE')
- Utiliser un mot de passe long (un mot de passe de 8 lettres est plus efficace d'un MdP de 4 lettres).

Pour l'attaquant :

- Créer un dictionnaire de mots de passe en les classant du plus probable au moins probable
- Essayer chaque mot de passe en commençant par le plus probable

**To do**

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
